# Kaggle Titanic

Будем решать задачу предсказания того, выжил пассажир Титаника или нет, по некоторым данным о пассажире.

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
%pylab inline

Загрузим данные и объединим train и test в один dataframe - так будет удобнее обрабатывать признаки.

In [ ]:
Train = pd.read_csv("train.csv")
Test = pd.read_csv("test.csv")
Test['Survived'] = -1

Data = pd.concat([Train,Test])

In [ ]:
len(Train), len(Test), len(Data)

# Обработка признаков

1) Найдем признаки, у которых есть пропущенные значения и обработаем эти пропущенные значения

2) Переведём категориальные признаки в числовые

Обработаем признак 'Sex'

In [ ]:
print(Data['Sex'].value_counts(dropna=False))

In [ ]:
Data['sex'] = Data.apply(lambda row: 1 if row.Sex == 'male' else 0, axis=1)

Посмотрим, как влияет пол человека и класс билета (Pclass) на выживаемость

In [ ]:
import seaborn as sns

grid = sns.FacetGrid(Data[Data.Survived!=-1], col = "Pclass", row = "Sex", hue = "Survived", \
                     palette = 'seismic')
grid = grid.map(plt.scatter, "PassengerId", "Age")
grid.add_legend()
grid

Обработаем признак 'Embarked'

In [ ]:
Data['Embarked'].value_counts(dropna=False)

In [ ]:
Data['EmbS'] = Data.apply(lambda row: 1 if row.Embarked == 'S' else 0, axis=1)
Data['EmbC'] = Data.apply(lambda row: 1 if row.Embarked == 'C' else 0, axis=1)
Data['EmbQ'] = Data.apply(lambda row: 1 if row.Embarked == 'Q' else 0, axis=1)

Обработаем признак 'Age'

1) Проверьте, если ли в Age пропущенные значения

2) Если пропущенные значения есть, заполните их медианным значением Age

In [ ]:
#есть ли Nanы в Age?
#Your code here

In [ ]:
Data.head()

In [ ]:
#Заполните пропущенные значения Age медианным значением
#Your code here

Различные способы визуализации данных упрощают понимание признаков и зависимостей между ними.

Посмотрим на распределение Pclass по возрастам.

In [ ]:
import seaborn as sns

Data[Data.Survived!=-1][['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean()
# plot age distribution by title
facet = sns.FacetGrid(data = Data[Data.Survived!=-1], hue = "Pclass", legend_out=True, size = 5)
facet = facet.map(sns.kdeplot, "Age")
facet.add_legend();

Обработаем признак 'Fare'

1) Проверьте, если ли в Fare пропущенные значения

2) Если пропущенные значения есть, заполните их медианным значением Fare

In [ ]:
#Your code here

Обработаем признак 'Pclass'

Есть ли в Pclass пропущенные значения? Если есть, обработайте их.

In [ ]:
#Your code here

Обработаем признак 'SibSp' (число братьев или сестер, мужей, жен)

Есть ли в SibSp пропущенные значения? Если есть, обработайте их.

In [ ]:
#Your code here

Обработаем признак 'Parch' (число родителей/детей)

Есть ли в Parch пропущенные значения? Если есть, обработайте их.

In [ ]:
#Your code here

# Разбиение Data на Train и Test и получение первого предсказания

1) оставьте в таблице Data только числовые признаки и назовите новую таблицу с признаками All

2) разбейте таблицу All на Xtrain, ytrain и Xtest.

In [ ]:
#your code here

Обучим RandomForest на очищенных данных и посмотрим на качество на кросс-валидации

In [ ]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=123, n_estimators=100, n_jobs=-1)
cross_val_score(rf, Xtrain, ytrain, cv=10, scoring='accuracy').mean()

Предскажем ответы на тесте и сделаем первый submit на Kaggle

1) сделаем предсказания и сохраним их в файл titanic_prediction_0.csv

2) загрузим этот файл на kaggle и посмотрим на результат

In [ ]:
rf.fit(Xtrain, ytrain)

pred_rf = rf.predict(Xtest)

Test['Survived']=pred_rf

Test[['PassengerId','Survived']].to_csv('titanic_prediction_26may_1.csv',index=False)

# Настройка параметров модели

In [ ]:
from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(RandomForestClassifier(random_state=123, n_jobs=-1),
                  param_grid={'n_estimators': np.arange(100,1100,100)},
                  cv=cv,
                  scoring='accuracy')
gs.fit(Xtrain, ytrain)
gs.best_score_, gs.best_estimator_, gs.best_params_

In [ ]:
gs = GridSearchCV(RandomForestClassifier(random_state=123, n_jobs=-1),
                  param_grid={'n_estimators': [800,900,1000],
                              'max_depth': [1,3,5,7,9,11]},
                  cv=cv,
                  scoring='accuracy')
gs.fit(Xtrain, ytrain)
gs.best_score_, gs.best_estimator_, gs.best_params_

In [ ]:
gs = GridSearchCV(RandomForestClassifier(n_estimators=900, max_depth=9, random_state=123, n_jobs=-1),
                  param_grid={'min_samples_split':[2,4,6],
                              'min_samples_leaf':[1,2,4,6]},
                  cv=cv,
                  scoring='accuracy')
gs.fit(Xtrain, ytrain)
gs.best_score_, gs.best_estimator_, gs.best_params_

Подберите с помощью GridSearchCV информационный критерий ('criterion': ['gini','entropy']). Используйте в RandomForestClassifier значения всех найденных параметров. 

In [ ]:
#Your code here

Посмотрим на важность признаков по мнению RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(n_estimators=900, max_depth=9, min_samples_split=2, \
                            min_samples_leaf=2, criterion='gini',
                            random_state=123, n_jobs=-1)
rf.fit(Xtrain, ytrain)

features = Xtrain.columns
importances = rf.feature_importances_
indices = np.argsort(importances)

plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), features[indices])
plt.xlabel('Relative Importance')
plt.show()

Сделаем второе предсказание с учётом настроенных параметров модели

1) обучите RandomForest с найденными параметрами и сделайте предсказание

2) отправьте его на Kaggle

In [ ]:
#Your code here

# Работа с признаками

Нарисуем матрицу корреляций признаков между собой и с целевой переменной

In [ ]:
import seaborn as sns

colormap = plt.cm.RdBu
plt.figure(figsize=(14,12))
plt.title('Pearson Correlation of Features', y=1.05, size=18)
sns.heatmap(All[All.Survived!=-1].astype(float).corr(),linewidths=0.1,vmax=1.0, #correlation matrix
            square=True, cmap=colormap, linecolor='white', annot=True)

In [ ]:
g = sns.pairplot(Data[Data.Survived!=-1][[u'Survived', u'Pclass', u'Sex', u'Age', u'Parch', u'Fare', u'Embarked']], \
                 hue='Survived', palette = 'seismic',size=4,diag_kind = 'kde',diag_kws=dict(shade=True),plot_kws=dict(s=50) )
g.set(xticklabels=[])

Бинаризуем признаки Age и Fare

* pd.cut - разбиение целочисленных данных на несколько интервалов по квантилям
* pd.qcut - разбиение числовых (не обязательно целочисленных) данных на несколько интервалов по квантилям

In [ ]:
Data['AgeBin'] = pd.cut(Data['Age'].astype(int), 5)

from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
Data['AgeBin_Code'] = label.fit_transform(Data['AgeBin'])

print(Data[['Age','AgeBin','AgeBin_Code']])

Бинаризуйте Fare, используя разбиение на 4 интервала

In [ ]:
#Your code here

Сконструируем новые признаки

1) Какую информацию можно извлечь из поля Name?

2) Какую информацию можно извлечь из поля Cabin?

3) Из матрицы корреляций мы видим, что признаки SibSp и Parch практически не коррелируют с таргетом. Какую информацию можно из них извлечь, исходя из смысла задачи?

In [ ]:
#Your code here

1) добавьте сконструированные признаки в таблицу All

2) нарисуйте новую матрицу корреляций

In [ ]:
#your code here

# Обучение и настройка параметров RandomForest для таблицы с добавленными признаками

1) Теперь в таблице All появились новые признаки. Заново разбейте её на Xtrain, ytrain и Xtest.

2) Найдите оптимальные параметры RandomForest для новых данных.

3) Сделайте предсказание, сохраните его в файл и отправьте на Kaggle.

In [ ]:
#Your code here

# Другие алгоритмы ML

# Градиентный бустинг

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

clf = GradientBoostingClassifier(random_state=123)
cross_val_score(clf, Xtrain, ytrain, cv=cv, scoring='accuracy').mean()

Подберите оптимальные параметры для градиентного бустинга

In [ ]:
#Your code here

In [ ]:
gb = GradientBoostingClassifier(learning_rate=..., n_estimators=..., max_depth=..., min_samples_split=..., min_samples_leaf=...,\
                                             random_state=123)
gb.fit(Xtrain, ytrain)

features = Xtrain.columns
importances = gb.feature_importances_
indices = np.argsort(importances)

plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), features[indices])
plt.xlabel('Relative Importance')
plt.show()

# AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(random_state=123)
cross_val_score(clf, Xtrain, ytrain, cv=cv, scoring='accuracy').mean()

Подберите n_estimaors и learning_rate для AdaBoost

In [ ]:
#Your code here

In [ ]:
ada = AdaBoostClassifier(learning_rate=..., n_estimators=..., random_state=123)
print cross_val_score(ada, Xtrain, ytrain, cv=cv, scoring='accuracy').mean()

ada.fit(Xtrain, ytrain)

# Extra Trees

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

clf = ExtraTreesClassifier(random_state=123, n_jobs=-1)
print cross_val_score(clf, Xtrain, ytrain, cv=cv, scoring='accuracy').mean()

Подберите оптимальные параметры для ExtraTrees (n_estimators, max_depth, min_samples_split, min_samples_leaf)

In [ ]:
#Your code here

In [ ]:
clf = ExtraTreesClassifier(n_estimators=..., max_depth=..., min_samples_split=..., min_samples_leaf=..., \
                           random_state=123, n_jobs=-1)
print cross_val_score(clf, Xtrain, ytrain, cv=cv, scoring='accuracy').mean()

Нарисуйте гистограмму важности признаков для ExtraTreesClassifier

In [ ]:
#Your code here

# Построение предсказания, использующего все натренированные алгоритмы

# Voting

In [ ]:
from sklearn.ensemble import VotingClassifier

voting = VotingClassifier(estimators=[('RF',rf),('GB',gb),('ADA',ada),('ET',et)], voting='soft')
voting.fit(Xtrain, ytrain)
pred_voting = voting.predict(XtestFull)

In [ ]:
cross_val_score(voting, Xtrain, ytrain, cv=cv, scoring='accuracy').mean()

Добавьте столбец с предсказанием в таблицу Test и сохраните предсказание в файл. Отправьте его на Kaggle.

In [ ]:
#Your code here

# Blending

Предскажите каждым алгоритмом не классы, а вероятности классов (predict_proba).

In [ ]:
rf_probas = ...
gb_probas = ...
ada_probas = ...
et_probas = ...

In [ ]:
blended_probas = 0.25 * (rf_probas+gb_probas+ada_probas+et_probas)

prediction = [1 if elem > 0.5 else 0 for elem in blended_probas]

In [ ]:
Test['Survived']=prediction

Test[['PassengerId','Survived']].to_csv('titanic_prediction_blending.csv',index=False)

# Stacking

Для того, чтобы сделать стэкинг, нам придется разбить Xtrain,ytrain на две части. На первой будем обучать алгоритмы первого уровня (RF, GB и т.д.), а на второй - обучать метаалгоритм, т.е. алгоритм, признаками для которого являются ответы алгоритмов первого уровня.

In [ ]:
Xtrain1, Xtrain2, ytrain1, ytrain2 = train_test_split(XtrainFull, ytrainfull, test_size=0.33)

Ещё раз переобучите все алгоритмы и настройте их параметры, но уже на уменьшенных данных - Xtrain1, ytrain1

Сделайте предсказания на Xtrain2 каждым из алгоритмов. Предсказывайте вероятности, а не классы (predict_proba)

In [ ]:
#Your code here

In [ ]:
SecondTrain = pd.DataFrame(np.nan, index=np.arange(len(Xtrain2)), columns=['RF','GB','ADA','ET'])

SecondTrain['RF'] = rf.predict(Xtrain2)
SecondTrain['GB'] = gb.predict(Xtrain2)
SecondTrain['ADA'] = ada.predict(Xtrain2)
SecondTrain['ET'] = et.predict(Xtrain2)

В качестве метаалгоритма будем использовать RandomForest. Подберите оптимальные параметры метаалгоритма.

In [ ]:
gs = GridSearchCV(RandomForestClassifier(random_state=111, n_jobs=-1),
                  param_grid={'n_estimators':np.arange(100,1100,100)},
                  cv=3,
                  scoring='accuracy')
gs.fit(SecondTrain, ytrain2)
gs.best_estimator_, gs.best_params_, gs.best_score_

In [ ]:
#Your code here

Сделаем предсказание нашего двухступенчатого алгоритма на Xtest.

In [ ]:
PredTest = pd.DataFrame(np.nan, index=np.arange(len(Xtest)), columns=['RF','GB','ADA','ET'])

PredTest['RF'] = rf.predict(Xtest)
PredTest['GB'] = gb.predict(Xtest)
PredTest['ADA'] = ada.predict(Xtest)
PredTest['ET'] = et.predict(Xtest)

pred_final = rf_final.predict(PredTest)

print len(Xtest), len(pred_final)

Сохраните предсказания в файл и отправьте файл на Kaggle.

In [ ]:
#Your code here

# Дальнейшие улучшения

1) Попробуйте использовать xgboost и catboost в качестве моделей первого уровня и/или второго уровня

2) Подумайте над признаками. Какие ещё факторы можно извлечь из данных?

3) Попробуйте снизить размерность пространства признаков: 

а) используйте feature_selection

b) используйте методы снижения размерности

4) Подумайте над другими способами кодирования категориальных признаков

In [ ]:
#Your code here